<a href="https://colab.research.google.com/github/jlonge4/gen_ai_utils/blob/main/csv_rag_retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Background:

When processing CSV's for RAG, a common processing technique to maintain order is to convert the row to a JSON format to maintain the relationships using column names as keys.

## Hypothesis

While necessary to store the full representation of the relationship structure as the content in your Document object for the model to use at query time, I hypothesize that removing the column names from the content for embedding(s) will maximize retrieval accuracy and distance between vectors (increase distinction among each record).

In [ ]:
!pip install sentence-transformers renumics-spotlight umap-learn plotly

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
model.to('cuda')

In [24]:
rows = [
    {"project_id": 1, "title": "Employee HelpDesk", "description": "Develop software to enhance and enable technicians to provide assistance to employees."},
    {"project_id": 2, "title": "Customer Support Chatbot", "description": "Create a chatbot that can answer customer questions and provide support."},
    {"project_id": 3, "title": "Automated Data Entry", "description": "Develop a system that can automatically enter data into a database from various sources."},
    {"project_id": 4, "title": "Inventory Management System", "description": "Create a system to track and manage inventory in a warehouse or store."},
    {"project_id": 5, "title": "Online Appointment Scheduling", "description": "Develop a platform that allows users to schedule appointments online."},
    {"project_id": 6, "title": "Expense Tracking Application", "description": "Create an app that helps users track and manage their expenses."},
    {"project_id": 7, "title": "Social Media Analytics Tool", "description": "Develop a tool that can analyze and provide insights on social media data."},
    {"project_id": 8, "title": "E-Learning Platform", "description": "Create a platform for online learning and education."},
    {"project_id": 9, "title": "Health and Fitness App", "description": "Develop an app that helps users track their health and fitness goals."},
    {"project_id": 10, "title": "Smart Home Automation System", "description": "Create a system that can automate and control various devices in a smart home."},
    {"project_id": 11, "title": "Online Marketplace Platform", "description": "Develop a platform for buying and selling goods and services online."}
]
row_strings = [str(row) for row in rows]
row_embeds = model.encode(row_strings)

In [25]:
row_embeds[0].shape

(384,)

In [26]:
q_embed = model.encode(["What is the description of project titled Automated Data Entry?"])

In [27]:
import numpy as np
import umap

def reduce_dim(ref_embeds, query_embed):
  embeddings = np.array([np.array(x) for x in ref_embeds])
  q_embed = np.array(query_embed)

  # Reduce dimensionality with UMAP
  reducer = umap.UMAP(n_neighbors=2, min_dist=0.1, n_components=2, random_state=42)
  refs_reduced = reducer.fit_transform(embeddings)
  query_reduced = reducer.transform(query_embed.reshape(1, -1))

  return refs_reduced, query_reduced

In [33]:
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd

def create_plot(refs_reduced, query_reduced):
    df_ref = pd.DataFrame({
        'x': refs_reduced[:, 0],
        'y': refs_reduced[:, 1],
        'type': ['Reference'] * len(refs_reduced)
    })

    df_query = pd.DataFrame({
        'x': [query_reduced[0, 0]],
        'y': [query_reduced[0, 1]],
        'type': ['Query']
    })

    # Concatenate the DataFrames
    df = pd.concat([df_ref, df_query], ignore_index=True)

    # Create the plot
    fig = px.scatter(df, x='x', y='y', color='type',
                    color_discrete_map={'Reference': 'red', 'Query': 'blue'},
                    title="UMAP Visualization of Text Embeddings with Query Highlighted",
                    labels={'x': 'UMAP Dimension 1', 'y': 'UMAP Dimension 2'},
                    hover_data={'x': False, 'y': False, 'type': True})

    # Update layout for better appearance
    fig.update_layout(
        plot_bgcolor='white',
        legend_title_text='Embedding Type',
        width=800,
        height=800,
        title={
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        }
    )

    # Update axes
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', zeroline=True, zerolinewidth=2, zerolinecolor='lightgrey')
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', zeroline=True, zerolinewidth=2, zerolinecolor='lightgrey')

    # Show the plot
    fig.show()

# Observation

Note that the scale fairly small here, the query is closer to most of the embeddings

In [34]:
refs_reduced, query_reduced = reduce_dim(row_embeds, q_embed)
create_plot(refs_reduced, query_reduced)

/usr/local/lib/python3.10/dist-packages/umap/umap_.py:1945: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.



In [35]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy

cosine_similarities_w_columns = cosine_similarity(q_embed, row_embeds)
closest_vectors_indices = numpy.argsort(cosine_similarities_w_columns[0])[-4:][::-1]
closest_vectors_scores = cosine_similarities_w_columns[0][closest_vectors_indices]

for i, idx in enumerate(closest_vectors_indices):
    print(f"Score: {closest_vectors_scores[i]}, Row: {rows[idx]}")

Score: 0.5482973456382751, Row: {'project_id': 3, 'title': 'Automated Data Entry', 'description': 'Develop a system that can automatically enter data into a database from various sources.'}
Score: 0.28474557399749756, Row: {'project_id': 1, 'title': 'Employee HelpDesk', 'description': 'Develop software to enhance and enable technicians to provide assistance to employees.'}
Score: 0.2832058370113373, Row: {'project_id': 10, 'title': 'Smart Home Automation System', 'description': 'Create a system that can automate and control various devices in a smart home.'}
Score: 0.25437596440315247, Row: {'project_id': 7, 'title': 'Social Media Analytics Tool', 'description': 'Develop a tool that can analyze and provide insights on social media data.'}


In [36]:
rows_wo_json = [str(str(row["project_id"]) + '; ' + row['title'] + '; ' + row["description"]) for row in rows]
row_embeds_2 = model.encode(rows_wo_json)
rows_wo_json[0]

'1; Employee HelpDesk; Develop software to enhance and enable technicians to provide assistance to employees.'

# Observation

Note that the scale is larger, and now the query is near only 4 other vectors.

In [37]:
refs_reduced_2, query_reduced_2 = reduce_dim(row_embeds_2, q_embed)
create_plot(refs_reduced_2, query_reduced_2)

/usr/local/lib/python3.10/dist-packages/umap/umap_.py:1945: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.



# Observation 2

Note that not only have the cosine similarities increased, but the rank of the results is now what we would expect (ie records mentioning the word automation are first and second)

In [43]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy

cosine_similarities_wo = cosine_similarity(q_embed, row_embeds_2)
closest_vectors_indices = numpy.argsort(cosine_similarities_wo[0])[-4:][::-1]
closest_vectors_scores = cosine_similarities_wo[0][closest_vectors_indices]

for i, idx in enumerate(closest_vectors_indices):
    print(f"Score: {closest_vectors_scores[i]}, Row: {rows_wo_json[idx]}")

Score: 0.7760639190673828, Row: 3; Automated Data Entry; Develop a system that can automatically enter data into a database from various sources.
Score: 0.43231236934661865, Row: 10; Smart Home Automation System; Create a system that can automate and control various devices in a smart home.
Score: 0.40793299674987793, Row: 4; Inventory Management System; Create a system to track and manage inventory in a warehouse or store.
Score: 0.3983978033065796, Row: 1; Employee HelpDesk; Develop software to enhance and enable technicians to provide assistance to employees.


In [44]:
from scipy.spatial.distance import cdist

distances = cdist(row_embeds, row_embeds, "cosine")
mean_distance_1 = np.mean(distances)
print(f"Average distance between vectors with column names included: {mean_distance_1}")

distances = cdist(row_embeds_2, row_embeds_2, "cosine")
mean_distance = np.mean(distances)
print(f"Average distance without columns: {mean_distance}")

Average distance between vectors with column names included: 0.31624439762906154
Average distance without columns: 0.596461488547043


SNR

In [42]:
def calculate_snr(signal, noise):
    mean_signal = np.mean(signal)
    snr = mean_signal / noise
    return snr

SNR is expected to be higher (signal more indistinguishable from the "noise") for records that include column names repeatedly throughout references.

In [45]:
snr_w_columns= calculate_snr(cosine_similarities_w_columns, mean_distance_1)
snr_wo_columns = calculate_snr(cosine_similarities_wo, mean_distance)

print(f"SNR with columns: {snr_w_columns}")
print(f"SNR without columns: {snr_wo_columns}")

SNR with columns: 0.77052802881513
SNR without columns: 0.6169394109580617


# Limitations

- Context matters: Sometimes, what appears to be redundancy might carry important contextual information.
- Domain-specific terminology: In specialized fields, apparent redundancy might be necessary for precision.
- Query diversity: The effectiveness of this approach might vary depending on the nature and diversity of potential queries.